### In Case of Using CoLab : 구글드라이브 연계

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


★ setting point: Set the folder path where your data is stored.

example: "/content/gdrive/MyDrive/Colab Notebooks/llama/01. data preparation/dataset"


In [20]:
dataPath = "/content/gdrive/MyDrive/LLMClass/datasets/"
#dataPath = "./dataset/"
!ls /content/gdrive/MyDrive/LLMStudy/datasets/

ls: cannot access '/content/gdrive/MyDrive/LLMStudy/datasets/': No such file or directory


### Install Library

In [3]:
# datasets: loads and manages data from hugging-face's community.
# jsonlines: handles json format data
!pip install datasets==2.16.1 jsonlines==4.0.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.10.0 which is incompatible.


### Login to access huggign face

In [4]:
# How to check HF access token: hugging face -> select account at top right -> settings -> access token
import huggingface_hub
huggingface_hub.login()

### Colab 에서 huggingface hub 로그인이 잘 안되면 아래와 같이 cli login 으로 진행

In [5]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write

In [ ]:
# Define installed libraries

In [6]:
import pandas as pd
import json
import jsonlines
from datasets import Dataset
import os

# ★ Hugging face 없이 csv 바로 튜닝데이터로 변환 ★ HF 호환

In [10]:
csvfilepath = dataPath+"btsdata_kor.csv"
jsonfilepath = dataPath+"btsdata_kor.json"
#csvfilepath = dataPath+"politec.csv"
#jsonfilepath = dataPath+"politec.json"

In [11]:
import pandas as pd

In [12]:
#csvdata = pd.read_csv(csvfilepath, encoding="ms949")
try:
    csvdata = pd.read_csv(csvfilepath, encoding="ms949")
except Exception as e:
    csvdata = pd.read_csv(csvfilepath, encoding="utf-8")

data_json = csvdata.to_json(jsonfilepath)

In [13]:
df = pd.read_json(jsonfilepath)

In [14]:
df.head(5)

,instruction,output
0,BTS의 대표 인사말은 무엇인가요?,"방! 탄! 안녕하세요, 방탄소년단입니다"
1,BTS는 누구인가요?,2013년 6월 13일에 데뷔한 빅히트 뮤직 소속의 7인조 대한민국 국적 보이그룹입니다.
2,BTS의 멤버 이름은 무엇인가요?,"김남준, 김석진, 민윤기, 정호석, 박지민, 김태형, 전정국입니다."
3,BTS의 데뷔 앨범은 무엇인가요?,2 COOL 4 SKOOL
4,BTS가 수상한 첫번째 국제적인 상은 무엇인가요?,2017년 BBMA 'Top Social Artist'상 입니다.


In [15]:
from datasets import Dataset

def format_example(row):
    return {
        'text': f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>

        {row['instruction']}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>

        {row['output']}<|eot_id|>"""
    }

# 판다스 데이터프레임을 데이터셋으로 변환하고, 포맷팅 함수 적용
dataset = Dataset.from_pandas(df)
dataset = dataset.map(format_example)

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

In [16]:
dataset[2]

{'instruction': 'BTS의 멤버 이름은 무엇인가요?',
 'output': '김남준, 김석진, 민윤기, 정호석, 박지민, 김태형, 전정국입니다.',
 '__index_level_0__': 2,
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\n        BTS의 멤버 이름은 무엇인가요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n        김남준, 김석진, 민윤기, 정호석, 박지민, 김태형, 전정국입니다.<|eot_id|>'}

In [19]:
# 필요시 허깅페이스에 업로드
dataset.push_to_hub("rssaem/btsdata_kor")
#dataset.push_to_hub("rssaem/llm_ko_politec")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/rssaem/btsdata_kor/commit/67f6645c128703a3701e4d9d2370f75a0a5adb12', commit_message='Upload dataset', commit_description='', oid='67f6645c128703a3701e4d9d2370f75a0a5adb12', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/rssaem/btsdata_kor', endpoint='https://huggingface.co', repo_type='dataset', repo_id='rssaem/btsdata_kor'), pr_revision=None, pr_num=None)